SELECT [DISTINCT]   
    &nbsp;&nbsp;&nbsp;&nbsp;Tablename1.columnname1,  
    &nbsp;&nbsp;&nbsp;&nbsp;Tablename2.columnname3,  
    &nbsp;&nbsp;&nbsp;&nbsp;.  
    &nbsp;&nbsp;&nbsp;&nbsp;.  
    &nbsp;&nbsp;&nbsp;&nbsp;TablenameX.columnnameY  
FROM Tablename1  
[LEFT] JOIN Tablename2   
    &nbsp;&nbsp;&nbsp;&nbsp;ON conditions  
[LEFT] JOIN Tablename3   
    &nbsp;&nbsp;&nbsp;&nbsp;ON conditions  
.  
.  
[LEFT] JOIN TablenameX  
    &nbsp;&nbsp;&nbsp;&nbsp;ON conditions  
WHERE 1=1  
    &nbsp;&nbsp;&nbsp;&nbsp;AND conditions  
[GROUP BY] TablenameN.columnnameM, ...  
[HAVING] aggregated conditions  
[ORDER BY] TablenameL.columnnameO, ...  
[LIMIT] number  

Мы имеем данные заказов ресторана(orders) и цены продуктов(products), которые загрузим из файлов CSV.

In [1]:
import pandas as pd
from pandasql import sqldf
pdsql = lambda q: sqldf(q, globals())

Считаем данные

In [2]:
orders = pd.read_csv(r'D:\python_work\datasets\SQL_data\SQL_restaurant-1-orders.csv')
orders.columns = ['number', 'timestamp', 'item', 'quantity', 'price', 'total_products_in_cart']
orders['timestamp'] = pd.to_datetime(orders['timestamp'])
orders['date'] = orders['timestamp'].dt.date 
orders.sample(2)

,number,timestamp,item,quantity,price,total_products_in_cart,date
41767,3466,2016-10-20 18:07:00,Chicken Chaat,1,4.95,4,2016-10-20
69206,7478,2017-11-21 20:10:00,Lamb Biryani,1,9.95,3,2017-11-21


In [3]:
# Проверим размер таблицы
orders.shape

(74818, 7)

In [4]:
# Проверим диапазон дат в наших данных
print(f"Timeline: {orders['date'].min()} to {orders['date'].max()}")

Timeline: 2015-01-09 to 2019-12-07


In [5]:
products = pd.read_csv(r'D:\python_work\datasets\SQL_data\SQL_restaurant-1-products-price.csv')
products.columns = ['item', 'price']
products.sample(2)

,item,price
166,Butter Chicken,9.95
132,Dhansak - Lamb,8.95


In [6]:
products.shape

(248, 2)

In [7]:
orders.item.nunique()

248

**JOIN**

Проверим, представляет ли orders.price цену одного товара или общую цену заказа?

In [8]:
result = pdsql(
"""
SELECT
    orders.*,
    products.price AS product_price
FROM orders
LEFT JOIN products 
    ON products.item = orders.item
""")

In [9]:
result.sample(3)

,number,timestamp,item,quantity,price,total_products_in_cart,date,product_price
54323,15666,2019-06-07 18:52:00.000000,Bhuna - Chicken,2,8.95,2,2019-06-07,8.95
44555,12197,2018-11-25 18:47:00.000000,Bombay Aloo,1,5.95,10,2018-11-25,5.95
63825,4238,2017-08-01 18:02:00.000000,Saag,1,8.95,5,2017-08-01,8.95


Видим, что orders.price здесь - это цена одного продукта.

**SELECT/WHERE**

Проверим продукты(уникальные), цена которых больше 10$. Результат отсортируем в алфавитном порядке

In [10]:
result = pdsql(
"""
SELECT
    DISTINCT(item)    
FROM orders
WHERE price > 10
ORDER BY item 
""")

In [11]:
result.head(5)

,item
0,Bengal Fish Biryani
1,Bengal Fish Karahi
2,Bengal Fry Fish
3,Bengal King Prawn
4,Bhuna


In [12]:
print(f'Количество продуктов, цена которых больше 10$ - {result.shape[0]}')

Количество продуктов, цена которых больше 10$ - 52


In [13]:
# Проверим результат
orders.query('price > 10').item.nunique()

52

**Aggregations**

Проверим общую сумму по каждому заказу. Отсортируем результат по убыванию суммы заказа

In [14]:
result = pdsql(
"""
SELECT
    number,
    SUM(price * quantity) AS total_sum
FROM orders
GROUP BY number
ORDER BY total_sum DESC
""")

In [15]:
result.head(5)

,number,total_sum
0,6769,1242.00
1,6768,685.25
2,15840,660.45
3,9412,582.00
4,9411,460.75


In [16]:
# Проверим сумму по заказу "6769" 
(orders.query('number == 6769').quantity * orders.query('number == 6769').price).sum()	 

1242.0

Сколько заказов размещалось ежедневно в 2019 году?

In [17]:
result = pdsql(
"""
SELECT 
    DATE(date) AS date,
    COUNT(DISTINCT number) AS num_orders
FROM orders
WHERE  date >= '2019-01-01'
   AND date < '2020-01-01'
GROUP BY date
""")

In [18]:
result

,date,num_orders
0,2019-01-01,14
1,2019-01-02,26
2,2019-01-03,32
3,2019-01-04,6
4,2019-01-05,14
...,...,...
210,2019-12-03,11
211,2019-12-04,25
212,2019-12-05,23
213,2019-12-06,9


In [19]:
# Проверим результат
orders.date = pd.to_datetime(orders.date)
orders[orders.date.dt.year == 2019].groupby('date')['number'].nunique()

date
2019-01-01    14
2019-01-02    26
2019-01-03    32
2019-01-04     6
2019-01-05    14
              ..
2019-12-03    11
2019-12-04    25
2019-12-05    23
2019-12-06     9
2019-12-07    18
Name: number, Length: 215, dtype: int64

**CASE Statements**

In [20]:
orders.head(1)

,number,timestamp,item,quantity,price,total_products_in_cart,date
0,16118,2019-03-08 20:25:00,Plain Papadum,2,0.8,6,2019-03-08


Категоризируем даты в 2019г. по количеству продаж:  
-high level: > 30 sales  
-medium level: 10-30 sales  
-low level: < 10 sales

In [22]:
result = pdsql(
"""
SELECT 
    date,
    num_orders,
    CASE WHEN num_orders > 30 THEN 'high'
         WHEN num_orders < 10 THEN 'low'
         ELSE 'medium'END AS category
FROM (
SELECT 
    DATE(date) AS date,
    COUNT(DISTINCT number) AS num_orders
FROM orders
WHERE  date >= '2019-01-01'
    AND date < '2020-01-01'
GROUP BY date
) AS q
""")

In [24]:
result.head(3)

,date,num_orders,category
0,2019-01-01,14,medium
1,2019-01-02,26,medium
2,2019-01-03,32,high


Реализация в pandas

In [47]:
categ = orders[orders.date.dt.year == 2019].groupby('date', as_index=False)['number'].nunique()\
        .rename(columns={'number':'num_orders'})

In [48]:
categ.head(1) 

,date,num_orders
0,2019-01-01,14


In [50]:
bins = [0, 9, 30, categ['num_orders'].max()]
labels = ["low", "medium", "high"]
categ['category'] = pd.cut(categ['num_orders'], bins=bins, labels=labels)

In [51]:
categ.head(3)

,date,num_orders,category
0,2019-01-01,14,medium
1,2019-01-02,26,medium
2,2019-01-03,32,high


Общие табличные выражения

In [52]:
result = pdsql(
"""
WITH daily_orders AS (
SELECT 
    DATE(date) AS date,
    COUNT(DISTINCT number) AS num_orders
FROM orders
WHERE date >= '2019-01-01'
    AND date < '2020-01-01'
GROUP BY date
)

SELECT 
    date,
    num_orders,
    CASE WHEN num_orders > 30 THEN 'high'
         WHEN num_orders < 10 THEN 'low'
         ELSE 'medium'END AS category
FROM daily_orders
""")

In [53]:
result.head(3)

,date,num_orders,category
0,2019-01-01,14,medium
1,2019-01-02,26,medium
2,2019-01-03,32,high


**WINDOW FUNCTIONS**

Определить три самых дорогих заказа на каждый день  
Шаг1: Получим общую стоимость всех заказов на каждый день

In [54]:
result = pdsql(
"""
SELECT
    DATE(date) AS date,
    number,
    SUM(price * quantity) AS total_sum
FROM orders
GROUP BY date, number
""")

In [55]:
result

,date,number,total_sum
0,2015-01-09,630,3.95
1,2015-01-10,1375,15.85
2,2015-01-10,1376,20.85
3,2015-01-10,1382,9.90
4,2015-01-10,1385,46.65
...,...,...,...
13392,2019-12-07,15753,24.80
13393,2019-12-07,15754,33.35
13394,2019-12-07,15755,36.75
13395,2019-12-07,15756,28.80


Шаг2: Проранжируем все заказы от самого дорогого - 1 к менее дорогим

In [59]:
result = pdsql(
"""
WITH orders_sum AS (
SELECT
    DATE(date) AS date,
    number,
    SUM(price * quantity) AS total_sum
FROM orders
GROUP BY date, number
)
SELECT 
    date,
    number,
    total_sum,
    ROW_NUMBER() OVER(PARTITION BY date ORDER BY total_sum DESC) AS ranking
FROM orders_sum
ORDER BY date, ranking 
""")

In [61]:
result.head(20)

,date,number,total_sum,ranking
0,2015-01-09,630,3.95,1
1,2015-01-10,1385,46.65,1
2,2015-01-10,1376,20.85,2
3,2015-01-10,1375,15.85,3
4,2015-01-10,1382,9.90,4
5,2015-01-10,1388,4.45,5
6,2015-01-10,1386,3.95,6
7,2015-01-10,1387,3.95,7
8,2015-02-10,1403,43.65,1
9,2015-02-10,1390,25.80,2


Шаг3: Получим ТОП-3 заказа на каждый день

In [62]:
result = pdsql(
"""
WITH orders_sum AS (
SELECT
    DATE(date) AS date,
    number,
    SUM(price * quantity) AS total_sum
FROM orders
GROUP BY date, number
)
SELECT *
FROM
(
SELECT 
    date,
    number,
    total_sum,
    ROW_NUMBER() OVER(PARTITION BY date ORDER BY total_sum DESC) AS ranking
FROM orders_sum
ORDER BY date, ranking 
)
WHERE ranking <= 3 
""")

In [63]:
result.head(10)

,date,number,total_sum,ranking
0,2015-01-09,630,3.95,1
1,2015-01-10,1385,46.65,1
2,2015-01-10,1376,20.85,2
3,2015-01-10,1375,15.85,3
4,2015-02-10,1403,43.65,1
5,2015-02-10,1390,25.80,2
6,2015-02-10,1402,24.80,3
7,2015-08-09,647,3.95,1
8,2015-08-09,648,3.95,2
9,2015-08-09,651,3.95,3
